
### Entrada
- x1: Local da Coleta em km
- x2: Nivel da Mare em m
- x3: Umidade do Ar (%)
- x4: Índice Pluvimétrico (mm)¹
- x5: Temperatura (°C)

### Saida
- y1: pH
- y2: Condutividade Elétrica (mS.cmˉ¹)
- y3: Oxigênio Dissolvido (mg.Lˉ¹)
- y4: Sólidos Totais Dissolvidos (ppm)


In [1]:
!pip install --upgrade pip
!pip install tensorflow 
!pip install openpyxl

In [2]:
import os

os.chdir("./content")

lm_dir = "tf-levenberg-marquardt"
if not os.path.exists(lm_dir):
  !git clone https://github.com/fabiodimarco/$lm_dir

os.chdir(lm_dir)

In [3]:
import numpy as np
import pandas as pd

def create_dataframe(output, label):
    df = pd.read_excel("../../../data.xlsx")
    x1, x2, x3, x4, x5  = df['x1'], df['x2'], df['x3'], df['x4'], df['x5']

    input = np.vstack([x1, x2, x3, x4, x5 ]).T
    output = np.array(df[output])
    return output, input

output, input = create_dataframe(output="y3", label="oxigenio")

In [4]:
import tensorflow as tf
import numpy as np
from keras import regularizers
from keras import initializers
import levenberg_marquardt as lm

# layers, neurons
class ShuffleArchitecture:
    def __init__(self, input_size, hidden_sizes, output_size, act_h, act_o, param_reg):
        self.input_size = input_size
        self.hidden_sizes = hidden_sizes
        self.output_size = output_size
        self.act_h = act_h
        self.act_o = act_o
        self.regularizer = regularizers.L2(param_reg)
        self.initializer = initializers.RandomUniform(minval=-0.5, maxval=0.5, seed=np.random.randint(1, 10000))

    def set_architecture(self):
        self.model = tf.keras.Sequential()
        self.model.add(tf.keras.layers.Dense(self.hidden_sizes[0],
                        input_shape=(self.input_size,),
                        activation=self.act_h,
                        kernel_regularizer=self.regularizer,
                        kernel_initializer=self.initializer,                        
                        ))  # input layer
        
        for size in self.hidden_sizes[1:]:  # hidden layers
            self.model.add(tf.keras.layers.Dense(size,
                            activation=self.act_h,
                            kernel_regularizer=self.regularizer,
                            kernel_initializer=self.initializer,  
                        ))

        self.model.add(tf.keras.layers.Dense(self.output_size,
                        activation=self.act_o,
                        kernel_regularizer=self.regularizer,
                        kernel_initializer=self.initializer,  
                        ))  # output layer

    def create_model(self, _learning_rate):
        self.model.compile(
            optimizer=tf.keras.optimizers.Adam(learning_rate=_learning_rate),
            loss=tf.keras.losses.MeanSquaredError())

        self.lm_model = lm.ModelWrapper(
            tf.keras.models.clone_model(self.model))

        self.lm_model.compile(
            optimizer=tf.keras.optimizers.SGD(learning_rate=_learning_rate),
            loss=lm.MeanSquaredError())
        return(self.lm_model)

2024-02-26 11:01:13.854668: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-02-26 11:01:16.628040: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-26 11:01:16.628143: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-26 11:01:17.072108: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-02-26 11:01:17.932164: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-02-26 11:01:17.932853: I tensorflow/core/platform/cpu_feature_guard.cc:1

In [5]:
import tensorflow as tf
import matplotlib.pyplot as plt
import time
from sklearn.model_selection import train_test_split
from keras.callbacks import EarlyStopping
from sklearn.metrics import r2_score, mean_squared_error


class TrainWithSmallDataset:
    def __init__(self, batch_size=1000):
        self.batch_size = batch_size
        self.betters = []
    
    def create_dataset(self, input, output):
      input = tf.expand_dims(tf.cast(input, tf.float32), axis=-1)
      output = tf.expand_dims(tf.cast(output, tf.float32), axis=-1)

      dataset = tf.data.Dataset.from_tensor_slices((input, output))
      dataset = dataset.shuffle(len(input))
      dataset = dataset.batch(self.batch_size).cache()
      dataset = dataset.prefetch(tf.data.experimental.AUTOTUNE)
      return (dataset, input, output)

    def split_dataset(self, input, output):
      input_train, input_vt, output_train, output_vt = train_test_split(input, output, test_size=0.3, shuffle = True)
      input_val, input_test, output_val, output_test = train_test_split(input_vt, output_vt, test_size=0.5, shuffle = True)

      self.train_dataset, self.train_input, self.train_output = self.create_dataset(input_train, output_train)
      self.val_dataset, self.val_input, self.val_output = self.create_dataset(input_val, output_val)
      self.test_dataset, self.test_input, self.test_output = self.create_dataset(input_test, output_test)
      self.vt_dataset, self.vt_input, self.vt_output = self.create_dataset(input_vt, output_vt)
      self.dataset, self.input, self.output = self.create_dataset(input, output)

      self._train = (input_train, output_train)
      self._vt = (input_vt, output_vt)
      self._val = (input_val, output_val)
      self._test = (input_test, output_test)
 
 
    
    def train_using_lm(self, train_dataset, epochs=1000):
      early_stopping_monitor = EarlyStopping(monitor='val_loss',
                                              patience=6,
                                              restore_best_weights=True)

      t2_start = time.perf_counter()
      self.results = self.lm_model.fit(train_dataset,
                                            epochs=epochs,
                                            validation_data=self.val_dataset,
                                            callbacks=[early_stopping_monitor],
                                            verbose=0)
      t2_stop = time.perf_counter()
      print("Elapsed time: ", t2_stop - t2_start)
      print ("Stopped at epoch: ", early_stopping_monitor.stopped_epoch)

    def get_metrics(self):
          self.test_prediction = self.lm_model.predict(self.test_input)

          pred = self.lm_model.predict(self.input).flatten()
          test_pred = self.test_prediction.flatten()
          val_pred = self.lm_model.predict(self.val_input).flatten()
          vt_pred = self.lm_model.predict(self.vt_input).flatten()

          r2 = r2_score(self.output, pred)
          r2_test = r2_score(self.test_output, test_pred)
          r2_val = r2_score(self.val_output, val_pred)
          r2_vt = r2_score(self.vt_output, vt_pred)

          mse = mean_squared_error(self.output, pred)
          mse_test = mean_squared_error(self.test_output, test_pred)
          mse_val = mean_squared_error(self.val_output, val_pred)
          mse_vt = mean_squared_error(self.vt_output, vt_pred)

          metrics = {
                          'r2': r2,
                          'r2_test': r2_test,
                          'r2_val': r2_val,
                          'r2_vt': r2_vt,
                          'mse': mse,
                          'mse_test': mse_test,
                          'mse_val': mse_val,
                          'mse_vt': mse_vt
                          }

          return metrics

In [6]:
from itertools import product
import pickle

class Tester:
  def __init__(self, input, output, run_times=500, dataset_run_times=10):
    self.run_times = run_times
    self.better_metrics = {}
    self.dataset_run_times = dataset_run_times
    self.input, self.output = input, output
  
  def setArchitecure(self, trainer, _hidden_sizes, _pg, _lr):
    shuffler = ShuffleArchitecture(input_size=5,
                                    hidden_sizes=_hidden_sizes,
                                    output_size=1,
                                    act_h='tanh',
                                    act_o='linear',
                                    param_reg=_pg)
    shuffler.set_architecture()    
    trainer.lm_model = shuffler.create_model(_lr)

  def Train(self, trainer, epochs=1000):
    trainer.train_using_lm(trainer.train_dataset, epochs=epochs)
    return(trainer.get_metrics(), trainer.lm_model)

  def SaveModelWeights(self, model, fileName):
    path = f"../models/{fileName}.keras"
    open(path,'w').close()
    model.save_weights(path) 

  def SaveDataset(self, trainer, fileName):
    path = f"../dataset/{fileName}.pkl" 
    with open(path, 'wb') as f:
      pickle.dump((trainer._train, trainer._vt, trainer._val, trainer._test), f)


  def LoopWeights(self, sort_by, boundarie, trainer, idx):
    better_model = 0
    save = False

    for i in range(self.run_times):
      print (f"+++++++++++ [{idx}] | {i + 1} ++++++++++++++++++")
      metrics, model = self.Train(trainer)
      if (metrics[sort_by] >= boundarie): # should be <= to descending metrics
        fileName = f"model_{idx}_{better_model}"
        self.SaveModelWeights(model, fileName)
        self.better_metrics[fileName] = metrics
        better_model += 1
        save = True
    
    return(save)

# Init
  def Loop(self, sort_by, boundarie, hidden_sizes, regularizers, learning_rate):
    trainer = TrainWithSmallDataset()

    for count, (hidden_size, reg, lr) in enumerate(product(hidden_sizes, regularizers, learning_rate), start=1):
      header =  f"Hidden Size={hidden_size}, regularizer={reg}, learning_rate={lr}"
      print(f"Testando combinacao{count}: {header}")
      self.setArchitecure(trainer, hidden_size, reg, lr)
      for j in range(self.dataset_run_times):
        trainer.split_dataset(self.input, self.output)
        if (self.LoopWeights(sort_by, boundarie, trainer, f"{count}_{j}") == True):
          self.SaveDataset(trainer, f"dataset_{count}_{j}")
          self.DisplayBetterResults(sort_by, header, f"{count}_{j}")
        self.better_metrics = {}

  def DisplayBetterResults(self, sort_by, header, dataset=0):
    df = pd.DataFrame.from_dict(self.better_metrics, orient='index')
    df = df.sort_values([sort_by])
    display(df)
    path = f'../results/metrics_{dataset}'
    df.to_excel(f"{path}.xlsx", index=True)
    print(f"DataFrame salvo em {path}")
    with open(f"{path}.txt", 'w') as arquivo:
      arquivo.write(header)

In [7]:
tester = Tester(input, output, run_times=25, dataset_run_times=10)

In [8]:
tester.Loop(sort_by='r2',
            boundarie=-0.5,
            hidden_sizes = [[15, 7], [30, 15], [30, 15, 7]],
            regularizers=[0.02, 0.1],
            learning_rate=[0.02, 0.1])

Testando combinacao1: Hidden Size=[15, 7], regularizer=0.02, learning_rate=0.02
+++++++++++ [1_0] | 1 ++++++++++++++++++
Elapsed time:  2.288786654999967
Stopped at epoch:  36
1/1 [==============================] - 0s 15ms/step
+++++++++++ [1_0] | 2 ++++++++++++++++++
Elapsed time:  1.5722518729999138
Stopped at epoch:  45
1/1 [==============================] - 0s 14ms/step
+++++++++++ [1_0] | 3 ++++++++++++++++++
Elapsed time:  0.5231606120005381
Stopped at epoch:  15
1/1 [==============================] - 0s 14ms/step
+++++++++++ [1_0] | 4 ++++++++++++++++++
Elapsed time:  0.42787154100005864
Stopped at epoch:  12
1/1 [==============================] - 0s 15ms/step
+++++++++++ [1_0] | 5 ++++++++++++++++++
Elapsed time:  0.9896428769998238
Stopped at epoch:  27
1/1 [==============================] - 0s 15ms/step
+++++++++++ [1_0] | 6 ++++++++++++++++++
Elapsed time:  0.4184401849997812
Stopped at epoch:  11
1/1 [==============================] - 0s 16ms/step
+++++++++++ [1_0] | 7 ++++

,r2,r2_test,r2_val,r2_vt,mse,mse_test,mse_val,mse_vt
model_1_0_0,-0.133836,-1.063392,-0.411899,-0.472410,1.254823,0.640620,2.813878,1.663329
model_1_0_1,0.551569,0.716384,0.424578,0.481007,0.496281,0.088054,1.146800,0.586287
model_1_0_3,0.593522,0.644459,0.361804,0.418425,0.449852,0.110385,1.271908,0.656984
model_1_0_2,0.594653,0.741212,0.481966,0.532264,0.448600,0.080346,1.032427,0.528385
model_1_0_10,0.614971,0.527179,0.345651,0.387952,0.426113,0.146796,1.304101,0.691408
model_1_0_11,0.615518,0.523885,0.348019,0.389438,0.425509,0.147819,1.299381,0.689730
model_1_0_12,0.616476,0.505263,0.351590,0.389693,0.424449,0.153600,1.292263,0.689442
model_1_0_9,0.622302,0.533899,0.373318,0.411897,0.418001,0.144710,1.248961,0.664359
model_1_0_7,0.624645,0.594832,0.379675,0.426042,0.415408,0.125792,1.236292,0.648380
model_1_0_17,0.624691,0.485139,0.380230,0.410542,0.415357,0.159848,1.235186,0.665890


DataFrame salvo em ../results/metrics_1_0
+++++++++++ [1_1] | 1 ++++++++++++++++++
Elapsed time:  0.2788971979998678
Stopped at epoch:  7
1/1 [==============================] - 0s 20ms/step
+++++++++++ [1_1] | 2 ++++++++++++++++++
Elapsed time:  0.25146347000008973
Stopped at epoch:  6
1/1 [==============================] - 0s 15ms/step
+++++++++++ [1_1] | 3 ++++++++++++++++++
Elapsed time:  0.24567906699940067
Stopped at epoch:  6
1/1 [==============================] - 0s 15ms/step
+++++++++++ [1_1] | 4 ++++++++++++++++++
Elapsed time:  0.3962451070001407
Stopped at epoch:  10
1/1 [==============================] - 0s 16ms/step
+++++++++++ [1_1] | 5 ++++++++++++++++++
Elapsed time:  0.45342489999984537
Stopped at epoch:  8
1/1 [==============================] - 0s 17ms/step
+++++++++++ [1_1] | 6 ++++++++++++++++++
Elapsed time:  0.8857087309997951
Stopped at epoch:  11
1/1 [==============================] - 0s 15ms/step
+++++++++++ [1_1] | 7 ++++++++++++++++++
Elapsed time:  0.3107967

,r2,r2_test,r2_val,r2_vt,mse,mse_test,mse_val,mse_vt
model_1_1_0,0.646876,0.917571,-1.749845,0.670777,0.390805,0.091712,0.350945,0.213704
model_1_1_1,0.666225,0.914998,-2.009297,0.644437,0.369390,0.094575,0.384057,0.230802
model_1_1_2,0.670726,0.910516,-2.045919,0.636981,0.364410,0.099562,0.388731,0.235641
model_1_1_3,0.675057,0.898194,-2.217759,0.609900,0.359617,0.113272,0.410661,0.253220
model_1_1_4,0.682060,0.905399,-2.351153,0.604097,0.351865,0.105256,0.427686,0.256987
model_1_1_7,0.692327,0.877630,-2.512580,0.563963,0.340504,0.136151,0.448288,0.283039
model_1_1_5,0.692726,0.903900,-2.320258,0.605595,0.340062,0.106923,0.423743,0.256014
model_1_1_6,0.694893,0.899394,-2.384604,0.595553,0.337664,0.111936,0.431955,0.262533
model_1_1_16,0.697538,0.896793,-5.377870,0.316247,0.334736,0.114830,0.813966,0.443835
model_1_1_8,0.697602,0.888544,-2.377217,0.586391,0.334666,0.124009,0.431012,0.268481


DataFrame salvo em ../results/metrics_1_1
+++++++++++ [1_2] | 1 ++++++++++++++++++
Elapsed time:  0.6181722139999692
Stopped at epoch:  9
1/1 [==============================] - 0s 15ms/step
+++++++++++ [1_2] | 2 ++++++++++++++++++
Elapsed time:  0.29135599799974443
Stopped at epoch:  7
1/1 [==============================] - 0s 15ms/step
+++++++++++ [1_2] | 3 ++++++++++++++++++
Elapsed time:  0.28625614999964455
Stopped at epoch:  7
1/1 [==============================] - 0s 17ms/step
+++++++++++ [1_2] | 4 ++++++++++++++++++
Elapsed time:  0.2478580580000198
Stopped at epoch:  6
1/1 [==============================] - 0s 18ms/step
+++++++++++ [1_2] | 5 ++++++++++++++++++
Elapsed time:  0.31298191499990935
Stopped at epoch:  8
1/1 [==============================] - 0s 21ms/step
+++++++++++ [1_2] | 6 ++++++++++++++++++
Elapsed time:  0.5121751890001178
Stopped at epoch:  12
1/1 [==============================] - 0s 16ms/step
+++++++++++ [1_2] | 7 ++++++++++++++++++
Elapsed time:  0.90558590

...

DataFrame salvo em ../results/metrics_12_7
+++++++++++ [12_8] | 1 ++++++++++++++++++
Elapsed time:  0.03489910000098462
Stopped at epoch:  0
1/1 [==============================] - 0s 14ms/step
+++++++++++ [12_8] | 2 ++++++++++++++++++
Elapsed time:  0.031045585001265863
Stopped at epoch:  0
1/1 [==============================] - 0s 15ms/step
+++++++++++ [12_8] | 3 ++++++++++++++++++
Elapsed time:  0.030242872000599164
Stopped at epoch:  0
1/1 [==============================] - 0s 15ms/step
+++++++++++ [12_8] | 4 ++++++++++++++++++
Elapsed time:  0.03156939700056682
Stopped at epoch:  0
1/1 [==============================] - 0s 15ms/step
+++++++++++ [12_8] | 5 ++++++++++++++++++
Elapsed time:  0.02850023599967244
Stopped at epoch:  0
1/1 [==============================] - 0s 16ms/step
+++++++++++ [12_8] | 6 ++++++++++++++++++
Elapsed time:  0.029433712999889394
Stopped at epoch:  0
1/1 [==============================] - 0s 15ms/step
+++++++++++ [12_8] | 7 ++++++++++++++++++
Elapsed time

,r2,r2_test,r2_val,r2_vt,mse,mse_test,mse_val,mse_vt
model_12_8_0,0.76759,0.896095,-1.48818,-0.122475,0.257209,0.049643,1.058658,0.524474
model_12_8_22,0.76759,0.896095,-1.48818,-0.122475,0.257209,0.049643,1.058658,0.524474
model_12_8_21,0.76759,0.896095,-1.48818,-0.122475,0.257209,0.049643,1.058658,0.524474
model_12_8_20,0.76759,0.896095,-1.48818,-0.122475,0.257209,0.049643,1.058658,0.524474
model_12_8_19,0.76759,0.896095,-1.48818,-0.122475,0.257209,0.049643,1.058658,0.524474
model_12_8_18,0.76759,0.896095,-1.48818,-0.122475,0.257209,0.049643,1.058658,0.524474
model_12_8_17,0.76759,0.896095,-1.48818,-0.122475,0.257209,0.049643,1.058658,0.524474
model_12_8_16,0.76759,0.896095,-1.48818,-0.122475,0.257209,0.049643,1.058658,0.524474
model_12_8_15,0.76759,0.896095,-1.48818,-0.122475,0.257209,0.049643,1.058658,0.524474
model_12_8_14,0.76759,0.896095,-1.48818,-0.122475,0.257209,0.049643,1.058658,0.524474


DataFrame salvo em ../results/metrics_12_8
+++++++++++ [12_9] | 1 ++++++++++++++++++
Elapsed time:  0.04312760399989202
Stopped at epoch:  0
1/1 [==============================] - 0s 15ms/step
+++++++++++ [12_9] | 2 ++++++++++++++++++
Elapsed time:  0.029948993998914375
Stopped at epoch:  0
1/1 [==============================] - 0s 14ms/step
+++++++++++ [12_9] | 3 ++++++++++++++++++
Elapsed time:  0.029711310999118723
Stopped at epoch:  0
1/1 [==============================] - 0s 14ms/step
+++++++++++ [12_9] | 4 ++++++++++++++++++
Elapsed time:  0.028788182000425877
Stopped at epoch:  0
1/1 [==============================] - 0s 15ms/step
+++++++++++ [12_9] | 5 ++++++++++++++++++
Elapsed time:  0.028752099000485032
Stopped at epoch:  0
1/1 [==============================] - 0s 14ms/step
+++++++++++ [12_9] | 6 ++++++++++++++++++
Elapsed time:  0.030204509999748552
Stopped at epoch:  0
1/1 [==============================] - 0s 14ms/step
+++++++++++ [12_9] | 7 ++++++++++++++++++
Elapsed ti

,r2,r2_test,r2_val,r2_vt,mse,mse_test,mse_val,mse_vt
model_12_9_0,0.76759,0.722239,0.933878,0.800495,0.257209,0.393814,0.061817,0.237581
model_12_9_22,0.76759,0.722239,0.933878,0.800495,0.257209,0.393814,0.061817,0.237581
model_12_9_21,0.76759,0.722239,0.933878,0.800495,0.257209,0.393814,0.061817,0.237581
model_12_9_20,0.76759,0.722239,0.933878,0.800495,0.257209,0.393814,0.061817,0.237581
model_12_9_19,0.76759,0.722239,0.933878,0.800495,0.257209,0.393814,0.061817,0.237581
model_12_9_18,0.76759,0.722239,0.933878,0.800495,0.257209,0.393814,0.061817,0.237581
model_12_9_17,0.76759,0.722239,0.933878,0.800495,0.257209,0.393814,0.061817,0.237581
model_12_9_16,0.76759,0.722239,0.933878,0.800495,0.257209,0.393814,0.061817,0.237581
model_12_9_15,0.76759,0.722239,0.933878,0.800495,0.257209,0.393814,0.061817,0.237581
model_12_9_14,0.76759,0.722239,0.933878,0.800495,0.257209,0.393814,0.061817,0.237581


DataFrame salvo em ../results/metrics_12_9
